In [1]:
import pandas as pd

In [2]:
import os
if os.environ.get("AITUEDATA"):
    print('set')
    data_directory = os.environ['AITUEDATA']
else:
    print('not set')
    #Eusebio
    # data_directory = "C:/Users/joaquin/Desktop/mem_aitue/Datos/"
    #Mini-Eusebio
    # data_directory = "C:/Users/jquin/Desktop/Memoria de Titulo/Aitue/Datos/"
    #PC-Mem
    data_directory = "C:/Users/Joaquin/Desktop/mem_aitue/Datos/"

not set


# 1.Select Data

In [3]:
cot_all = pd.read_excel(data_directory+"cotizaciones_all.xlsx")
neg_all = pd.read_excel(data_directory+"negocios_all.xlsx")

In [4]:
cot_all.columns

Index(['ID', 'Fecha Cotizacion', 'Medio', 'Tipo de Medio', 'Proyecto', 'Etapa',
       'Productos', 'Presencial', 'Remoto', 'Jefe Comercial',
       'Ejecutivo Comercial', 'Total Productos', 'Descuentos',
       'Valor Final Venta', 'RUT', 'Nombre', 'Apellido 1', 'Apellido 2',
       'Nombre Completo', 'Telefono', 'Celular', 'Direccion', 'Region',
       'Provincia', 'Comuna', 'Correo Electronico', 'Sexo', 'Estado Civil',
       'Rango Edad', 'Fecha Nacimiento', 'Profesion'],
      dtype='object')

In [5]:
s_columns = ['ID', 'Fecha Cotizacion', 'Medio', 'Proyecto', 'Etapa','Productos', 'Presencial',
             'Total Productos','Descuentos','Valor Final Venta', 'RUT', 'Nombre', 'Apellido 1', 'Apellido 2',
             'Nombre Completo', 'Telefono', 'Celular', 'Direccion', 'Region','Provincia', 'Comuna', 
             'Correo Electronico', 'Sexo', 'Estado Civil','Rango Edad', 'Fecha Nacimiento', 'Profesion']

cot_all = cot_all[s_columns]

# 2.Clean Data

In [6]:
# variables a impiar y chequear

# Limpiar variable fecha de nacimiento de alguna manera
# Limpiar variable Sexo usando rutificador
# Limpar variables Direccion usando en lo posible el rutificador y otras fuentes

# 3.Construct Data

## 3.1 Nuevos Atributos

### 3.1.1.Edad
Variable construida a partir de la fecha de nacimiento

In [7]:
# Construir variable 'Edad' a partir de fecha de Nacimiento
import datetime
now = datetime.datetime.now().year

fechas_nacimiento = pd.to_datetime(cot_all['Fecha Nacimiento']).apply(lambda x: x.date()).tolist()

def anio_nac(x):
    try:
        year = int(x.year)
        dif = now - year
        if dif >= 18:
            return dif
        else:
            return 0
    except (ValueError,AttributeError):
        return 0
        
    return r

anio_nacimiento = [anio_nac(x) for x in fechas_nacimiento]
cot_all['edad'] = anio_nacimiento


### 3.1.2.Descuento
Variable que indica si el cliente recibio descuento

In [8]:
# Construir 'descuento': binario : [True, False]
recibio_descuento = cot_all['Descuentos'].tolist()

def is_desc(x):
    try:
        desc = int(x)
        if desc > 0:
            return True
        else:
            return False
    except ValueError:
        return False

descuento = [is_desc(x) for x in recibio_descuento]
cot_all['recibio_descuento'] = descuento
# descuento

### 3.1.3.Cantidad de Productos

In [9]:
# Construir 'productos': categorico : ['depto','casa','estacionamiento','bodega','Estudio']
productos = cot_all['Productos'].tolist()
depto = []; estacionamiento=[]; bodega = []; estudio = [];nan = 0
def get_products(x,word,numeric=False):
    w=0
    try:
        p =str(x)
        arr_p = x.split(',')
        for e in arr_p:
            if word in e:
                w+=1
#                 print(word + " :",e)
                break
            elif numeric and ('B' not in e or 'Est' not in e):
                try:
                    int(e)
#                     print(word + " :",e)
                    w+=1
                except ValueError:
                    pass
        return w
    except (ValueError,AttributeError):
        try:
            int(x)
            if numeric:
                return 1
            else:
                return 0
        except ValueError:
            return 0

cot_all['#vivienda'] = [get_products(x,'T',numeric=True) for x in productos]
cot_all['#bodega'] = [get_products(x,'Bod') for x in productos]
cot_all['#estacionamiento'] = [get_products(x,'Est') for x in productos]
cot_all['#estudio'] = [get_products(x,'Estudio') for x in productos]
cot_all['#lan'] = [get_products(x,'lan') for x in productos]

In [10]:
print(cot_all[['Productos','#vivienda','#bodega','#estacionamiento','#estudio','#lan']].head())
print(cot_all[['Productos','#vivienda','#bodega','#estacionamiento','#estudio','#lan']].tail())

  Productos  #vivienda  #bodega  #estacionamiento  #estudio  #lan
0    TA-103          1        0                 0         0     0
1    TA-103          1        0                 0         0     0
2    TA-102          1        0                 0         0     0
3    TA-104          1        0                 0         0     0
4    TA-103          1        0                 0         0     0
        Productos  #vivienda  #bodega  #estacionamiento  #estudio  #lan
2678          306          1        0                 0         0     0
2679         1111          1        0                 0         0     0
2680  906,Lan-906          1        0                 0         0     0
2681          912          1        0                 0         0     0
2682  309,Est-002          1        0                 1         0     0


### 3.1.4.Latitud y Longitud

In [11]:
# Constuir 'latitud' a partir de 'Direccion'
# Constuir 'longitud' a partir de 'Direccion'

### 3.1.5.Email
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [12]:
cot_all['has_correo'] = ~cot_all['Correo Electronico'].isnull()
cot_all[cot_all['has_correo'] == False][['has_correo','Correo Electronico']]

,has_correo,Correo Electronico


### 3.1.6.Region
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [13]:
cot_all['Region'].unique()

array(['VIII Región del Bío-Bío', 'II Región de Antofagasta',
       'IV Región de Coquimbo', 'VII Región del Maule',
       'X Región de Los Lagos', 'XIII Región Metropolitana de Santiago',
       'V Región de Valparaíso',
       'VI Región del Libertador General Bernardo O Higgins',
       'XIV Región de Los Ríos',
       'XI Región de Aysen del General Carlos Ibáñez del Campo',
       'I Región de Tarapacá', 'IX Región de La Araucanía',
       'XII Región de Magallanes y de La Antártica Chilena',
       'XV Region de Arica y Parinacota', 'III Región de Atacama'],
      dtype=object)

In [14]:
# Construir 'region': categorico : ['Bio-Bio','Santiago','Otro']

# reg = cot_all['Region']
# regiones = []

# def get_region(r):
#     if 'Bío-Bío' in r:
#         return 'Bio-Bio'
#     elif 'Santiago' in r:
#         return 'Santiago'
#     else:
#         return 'Otro'
        
# regiones = [ get_region(x) for x in reg]
# cot_all['region'] = regiones
# cot_all['region'].astype('category').head()
cot_all['region'] = cot_all['Region'].copy()
cot_all['region'] = cot_all['region'].replace(
    {'VIII Región del Bío-Bío':'Bio-Bio',
     'XIII Región Metropolitana de Santiago':'Metropolitana'}
)
cot_all['region'] = cot_all['region'].replace(
    ['II Región de Antofagasta',
       'IV Región de Coquimbo', 'VII Región del Maule',
       'X Región de Los Lagos',
       'V Región de Valparaíso',
       'VI Región del Libertador General Bernardo O Higgins',
       'XIV Región de Los Ríos',
       'XI Región de Aysen del General Carlos Ibáñez del Campo',
       'I Región de Tarapacá', 'IX Región de La Araucanía',
       'XII Región de Magallanes y de La Antártica Chilena',
       'XV Region de Arica y Parinacota', 'III Región de Atacama'],'Otro'
)

In [15]:
cot_all['region'].astype('category').head()

0    Bio-Bio
1    Bio-Bio
2    Bio-Bio
3    Bio-Bio
4    Bio-Bio
Name: region, dtype: category
Categories (3, object): [Bio-Bio, Metropolitana, Otro]

### 3.1.7.Provincia
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [16]:
cot_all['Provincia'].unique()

array(['Concepción', 'Antofagasta', 'Limarí', 'Talca', 'Biobío', 'Ñuble',
       'Llanquihue', 'Santiago', 'Maipo', 'Valparaíso', 'Cachapoal',
       'Arauco', 'Valdivia', 'Coyhaique', 'Linares', 'Osorno',
       'San Felipe de Aconcagua', 'Iquique', 'Cautín', 'Capitán Prat',
       'Aysén', 'Chacabuco', 'Melipilla', 'Magallanes', 'Malleco',
       'Arica ', 'Elqui', 'El Loa', 'Cauquenes', 'Colchagua', 'Valdivia ',
       'Copiapó', 'Última Esperanza', 'Cordillera', 'Curicó', 'Chiloé',
       'Choapa', 'Huasco'], dtype=object)

In [17]:
cot_all['provincia'] = cot_all['Provincia'].copy()
cot_all['provincia'] = cot_all['provincia'].replace(
    {'Concepción':'Concepcion',
     'Santiago':'Santiago'}
)
cot_all['provincia'] = cot_all['provincia'].replace(
    ['Antofagasta', 'Limarí', 'Talca', 'Biobío', 'Ñuble',
       'Llanquihue','Maipo', 'Valparaíso', 'Cachapoal',
       'Arauco', 'Valdivia', 'Coyhaique', 'Linares', 'Osorno',
       'San Felipe de Aconcagua', 'Iquique', 'Cautín', 'Capitán Prat',
       'Aysén', 'Chacabuco', 'Melipilla', 'Magallanes', 'Malleco',
       'Arica ', 'Elqui', 'El Loa', 'Cauquenes', 'Colchagua', 'Valdivia ',
       'Copiapó', 'Última Esperanza', 'Cordillera', 'Curicó', 'Chiloé',
       'Choapa', 'Huasco'],'Otro'
)

In [18]:
cot_all['provincia'].astype('category').head()

0    Concepcion
1    Concepcion
2    Concepcion
3    Concepcion
4    Concepcion
Name: provincia, dtype: category
Categories (3, object): [Concepcion, Otro, Santiago]

In [19]:
# # Construir 'provincia': categorico : ['Concepcion','Bio-Bio','Santiago','Otro']
# reg = cot_all['Provincia']

# def get_provincia(r):
#     if 'Concepción' in r:
#         return 'Concepcion'
#     elif 'Santiago' in r:
#         return 'Santiago'
#     else:
#         return 'Otro'
        
# regiones = [ get_provincia(x) for x in reg]
# cot_all['provincia'] = regiones
# cot_all['provincia'].astype('category').head()

### 3.1.8.Comuna
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [20]:
cot_all['Comuna'].unique()

array(['Concepción', 'Antofagasta', 'Ovalle', 'Constitución',
       'Talcahuano', 'Tomé', 'Hualpén', 'Los Ángeles', 'Chillán',
       'Yungay', 'Puerto Montt', 'Santiago', 'El Bosque', 'La Florida',
       'Las Condes', 'Ñuñoa', 'Providencia', 'Buin', 'Chiguayante',
       'Florida', 'Penco', 'San Pedro de la Paz', 'Viña del Mar',
       'Machalí', 'Vitacura', 'Arauco', 'Coronel', 'Contulmo', 'Concón',
       'Santa Juana', 'Puerto Varas', 'Cañete', 'Huechuraba', 'Valdivia',
       'Coyhaique', 'Colbún', 'Osorno', 'San Felipe', 'Iquique',
       'Rancagua', 'Lota', 'Temuco', 'Cochrane', 'Chillán Viejo', 'Maipú',
       'Aysén', 'Villarrica', 'La Reina', 'Peñalolén', 'Estación Central',
       'Colina', 'Quilicura', 'Curanilahue', 'Algarrobo', 'Talca',
       'San Carlos', 'Punta Arenas', 'Mulchén', 'Coelemu', 'Angol',
       'Laja', 'Nacimiento', 'Arica', 'La Serena', 'Hualqui',
       'San Miguel', 'Calama', 'Pelluhue', 'Quillón', 'Valparaíso',
       'Llanquihue', 'Los Álamos', 'Ren

In [21]:
cot_all['comuna'] = cot_all['Comuna'].copy()

cot_all['comuna'] = cot_all['comuna'].replace(
    ['Ovalle', 'Constitución','Tomé', 'Chillán','Yungay',
     'Puerto Montt', 'Santiago', 'El Bosque', 'La Florida','Las Condes',
     'Ñuñoa', 'Providencia', 'Buin','Florida','Viña del Mar',
     'Machalí', 'Vitacura', 'Arauco', 'Contulmo', 'Concón',
     'Santa Juana', 'Puerto Varas', 'Cañete', 'Huechuraba', 'Valdivia',
     'Coyhaique', 'Colbún', 'Osorno', 'San Felipe', 'Iquique',
     'Rancagua', 'Lota', 'Temuco', 'Cochrane', 'Chillán Viejo', 'Maipú',
     'Aysén', 'Villarrica', 'La Reina', 'Peñalolén', 'Estación Central',
     'Colina', 'Quilicura', 'Curanilahue', 'Algarrobo', 'Talca',
     'San Carlos', 'Punta Arenas', 'Mulchén', 'Coelemu', 'Angol',
     'Laja', 'Nacimiento', 'Arica', 'La Serena', 'Hualqui',
     'San Miguel', 'Calama', 'Pelluhue', 'Quillón', 'Valparaíso',
     'Llanquihue', 'Los Álamos', 'Renca', 'Santa Cruz', 'Panguipulli',
     'Cabrero', 'Cerrillos', 'Pudahuel', 'Copiapó', 'Natales', 'Tirúa',
     'Cauquenes', 'La Cisterna', 'Yumbel', 'Ránquil', 'Padre Las Casas',
     'Las Cabras', 'Coquimbo', 'Lebu', 'San José de Maipo', 'San Ramón',
     'Curicó', 'Independencia', 'Lampa ', 'Castro', 'Punitaqui',
     'Conchalí', 'San Rosendo', 'Alto Hospicio', 'Illapel', 'Huasco',
     'Collipulli', 'Tucapel', 'Alto Biobío'],'Otro'
)

In [22]:
cot_all['comuna'].astype('category').head()

0    Concepción
1    Concepción
2    Concepción
3    Concepción
4    Concepción
Name: comuna, dtype: category
Categories (10, object): [Antofagasta, Chiguayante, Concepción, Coronel, ..., Otro, Penco, San Pedro de la Paz, Talcahuano]

In [23]:
# Construir 'comuna': categorico : ['Concepcion','San Pedro','Talcahuano','Hualpen','Chiguayante','Santiago','Otro']
# reg = cot_all['Comuna']
# regiones = []

# def get_comuna(r):
#     if 'Concepción' in r:
#         return 'Concepcion'
#     elif 'Santiago' in r:
#         return 'Santiago'
#     elif 'San Pedro de la Paz' in r:
#         return 'San Pedro'
#     elif 'Talcahuano' in r:
#         return 'Talcahuano'
#     elif 'Chiguayante' in r:
#         return 'Chiguayante'
#     else:
#         return 'Otro'
        
# regiones = [ get_comuna(x) for x in reg]
# cot_all['comuna'] = regiones
# cot_all['comuna'].astype('category').head()

In [24]:
cot_all.columns

Index(['ID', 'Fecha Cotizacion', 'Medio', 'Proyecto', 'Etapa', 'Productos',
       'Presencial', 'Total Productos', 'Descuentos', 'Valor Final Venta',
       'RUT', 'Nombre', 'Apellido 1', 'Apellido 2', 'Nombre Completo',
       'Telefono', 'Celular', 'Direccion', 'Region', 'Provincia', 'Comuna',
       'Correo Electronico', 'Sexo', 'Estado Civil', 'Rango Edad',
       'Fecha Nacimiento', 'Profesion', 'edad', 'recibio_descuento',
       '#vivienda', '#bodega', '#estacionamiento', '#estudio', '#lan',
       'has_correo', 'region', 'provincia', 'comuna'],
      dtype='object')

### 3.1.9.Atributos Provistos
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [26]:
cot_all['is_nombre'] = ~cot_all['Nombre'].isnull()
cot_all['is_apellido1'] = ~cot_all['Apellido 1'].isnull()
cot_all['is_apellido2'] = ~cot_all['Apellido 2'].isnull()
cot_all['is_nombre_completo'] = ~cot_all['Nombre Completo'].isnull()

### 3.1.9.Cantidad de Cotizaciones
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [ ]:
# Construir '#cotizaciones' a partir del nro de filas donde participa el mismo rut
filas_ruts = []
cot_ruts = pd.DataFrame()
for group, frame in  cot_all.groupby('RUT'):
    rut_count = frame['ID'].count()
    filas_ruts.append(rut_count)
    print(frame['Region'].tolist())

### 3.1.10.Tiempo entre Cotizaciones
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [ ]:
# Construir 'promedio_de_tiempo' entre cotizacones donde participa el mismo rut

### 3.1.11.Negocio
Indica si el cliente siguio el proceso de cotizacion y lo transformo en un proceso de negocio

In [ ]:
# Construir '#Negocio' si el rut participo en la tabla Negocios. 
# ruts de los clientes en negocio
neg_ruts = neg_all['RUT'].unique().tolist()

cot_neg = cot_all.copy()
cot_neg['Negocio'] = cot_all['RUT'].isin(neg_ruts)

print(neg_all.shape[0])
print("Nro Filas en cot que tienen ruts en neg :",cot_neg.loc[cot_neg['Negocio']==True].shape[0])

# 4.Integrate Data

In [ ]:
# Transformar todo en un data set

# 5.Format Data